In [ ]:
%run setup_environment.py
import seq2fitness_models as models
import seq2fitness_traintools as traintools
import seq2fitness_train as train
import seq2fitness_protein_optimizer as protein_optimizer
import os
import pickle
import time
import gc
import joblib
from pprint import pprint
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## ESM Only

In [ ]:
esm_score_weights = {'mutant': 1.0,
                     'reference': 0.0
    } # Use mutant score only
model_params = {
    'ref_seq': ref_seq_amylase, # in the model checkpoint params
    'esm_modelname': 'esm2_t33_650M_UR50D',
    'esm_score_weights': esm_score_weights
}
optimizer_params = {
    'seqs_per_iter': 500, # typically 1000
    'num_iter': 10, # typically 250 or more. But 100 is enough for a test
    'T': 1.5, # 10 when not normalizing scores
    'score_threshold': 14.5,
    'seed': 7,
    'gamma': 1.0, # typically 1.0
    'cooling_rate': 0.9, # for slower simple cooling
    'num_mutations': 2,
    'sites_to_ignore': [1] # known bad sites that we should not sample sequences from
}

# Load from the pickle file
with open('single_mutant_scores_amylase_esm_only.pkl', 'rb') as file:
    initial_score_matrix = pickle.load(file)
optimizer = protein_optimizer.ProteinOptimizer(optimizer_params, model_params=model_params, esm_only=True,
                 score_matrix=initial_score_matrix,
                                                simple_simulated_annealing=False, cool_then_heat = False)
initial_score_matrix = optimizer.initial_score_matrix
print(f"phase transition threshold is {optimizer.score_threshold}")

In [ ]:
import matplotlib.pyplot as plt

# Assuming initial_score_matrix is already defined and is a NumPy array
# Flatten the matrix to get a 1D array
flattened_scores = initial_score_matrix.flatten()

# Plot the histogram with 60 bins
plt.hist(flattened_scores, bins=60, edgecolor='black')

# Add title and labels
plt.title('Histogram of Initial Score Matrix')
plt.xlabel('Scores')
plt.ylabel('Frequency')

# Show the plot
plt.show()


In [ ]:
start_time = time.time()
df, df_stats = optimizer.optimize()
end_time = time.time()

In [ ]:
optimizer.plot_scores()

In [ ]:
n_steps = optimizer_params['num_iter']*optimizer_params['seqs_per_iter']
n_mut = optimizer_params['num_mutations']
seed = optimizer_params['seed']
fn = f"protein_optimizer_results_num_mutations_{n_mut}_nsteps_{n_steps}_seed_{seed}_n_seqs_to_keep_{len(df)}.csv"
optimizer.save_results(filename=fn, n_seqs_to_keep=None)

In [ ]:
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
torch.cuda.empty_cache()
gc.collect()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is already loaded with the given structure

# Compute the percentiles
percentiles = np.percentile(df['ml_score'], [25, 50, 75, 90, 99])

# Find the best sequence and its score
best_score = df['ml_score'].max()
best_sequence = df.loc[df['ml_score'].idxmax(), 'sequences']

# Print the unique sequences found
print(f"There were {len(set(df['sequences']))} unique sequences found.")

# Print the best sequence and its score
print(f"Best Sequence:")
print(best_sequence)

print(f"Best Score: {best_score:.4f}")

# Print the percentiles
print(f"Percentiles:")
print(f"  25th Percentile: {percentiles[0]:.4f}")
print(f"  50th Percentile: {percentiles[1]:.4f}")
print(f"  75th Percentile: {percentiles[2]:.4f}")
print(f"  90th Percentile: {percentiles[3]:.4f}")
print(f"  99th Percentile: {percentiles[4]:.4f}")

# Plotting the percentiles
#plt.figure(figsize=(10, 6))
#plt.plot([25, 50, 75, 90, 99], percentiles, marker='o', linestyle='--', color='b')
#plt.xlabel('Percentile')
#plt.ylabel('Score')
#plt.title('Percentiles of Scores')
#plt.grid(True)
#plt.show()

# Plot the score distribution of the top 10,000 sequences
top_n = 20000
top_10000_df = df.nlargest(top_n, 'ml_score')

plt.figure(figsize=(10, 6))
plt.hist(top_10000_df['ml_score'], bins=50, alpha=0.75, color='g', edgecolor='black')
plt.axvline(best_score, color='r', linestyle='dashed', linewidth=1, label=f'Best Score: {best_score:.4f}')
plt.axvline(optimizer.wt_score, color='k', linestyle='dashed', linewidth=1, label=f'WT Score: {float(optimizer.wt_score):.4f}')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.title(f'Distribution of Scores (Top {top_n:,} Sequences)')
plt.legend()
plt.grid(True)
plt.savefig(f'Distribution of Scores (Top {top_n:,} Sequences)', bbox_inches='tight', dpi=200)
plt.show()

## ML Model Scores

In [ ]:
model_path = '../trained_models/Seq2Fitness_AAmylsase_DEMLPaper__modelrank_1_epoch_45_val_loss_0.5990.pt'
new_task_weights = {'fitness_dp3_activity': 2.0,
               'fitness_stain_removal': 1.0
    }

optimizer_params = {
    'new_task_weights': new_task_weights,
    'normalize_scores': True,
    'score_threshold': 3.5,
    'reversal_threshold': 0.0,
    #'ref_score_value': 0.0, # for energy normalization
    #'ref_score_scale': 0.0, # for energy normalization
    'seqs_per_iter': 520, # typically 1000
    'num_iter': 200, 
    'init_score_batch_size': 500, # only useful if score matrix is not specified, to compute single mutant scores
    'T': 1.5, # use 3 for not normalized scores
    'seed': 7,
    'gamma': 1.0, 
    'cooling_rate': 0.93, 
    'num_mutations': 5,
    'sample_variety_of_mutation_numbers': False,
    # 'num_sequences_proportions': [0, 1, 1, 1, 1, 1], 
    'sites_to_ignore': [1] # known bad sites that we should not sample sequences from
}
# Load from the pickle file
with open('single_mutant_scores_amylase.pkl', 'rb') as file:
   initial_score_matrix = pickle.load(file)

optimizer = protein_optimizer.ProteinOptimizer(optimizer_params, model_checkpoint_path =  model_path, model_params=None, esm_only=False,
                 score_matrix=initial_score_matrix,
                                                simple_simulated_annealing=False, cool_then_heat = False)
initial_score_matrix = optimizer.initial_score_matrix
print(f"phase transition threshold is {optimizer.score_threshold}")

In [ ]:
import matplotlib.pyplot as plt

# Assuming initial_score_matrix is already defined and is a NumPy array
# Flatten the matrix to get a 1D array
#flattened_scores = initial_score_matrix.flatten()/optimizer.wt_score
flattened_scores = initial_score_matrix.flatten()
# Plot the histogram with 60 bins
plt.hist(flattened_scores, bins=60, edgecolor='black')

# Add title and labels
plt.title('Histogram of Initial Score Matrix')
plt.xlabel('Scores')
plt.ylabel('Frequency')

# Show the plot
plt.show()


In [ ]:
start_time = time.time()
df, df_stats = optimizer.optimize()
end_time = time.time()
# Print the elapsed time

In [ ]:
optimizer.plot_scores()

In [ ]:
n_steps = optimizer_params['num_iter']*optimizer_params['seqs_per_iter']
n_mut = optimizer_params['num_mutations']
seed = optimizer_params['seed']
fn = f"protein_optimizer_results_num_mutations_{n_mut}_nsteps_{n_steps}_seed_{seed}_n_seqs_to_keep_{len(df)}.csv"
optimizer.save_results(filename=fn, n_seqs_to_keep=None)

In [ ]:
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
torch.cuda.empty_cache()
gc.collect()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is already loaded with the given structure

# Compute the percentiles
percentiles = np.percentile(df['ml_score'], [25, 50, 75, 90, 99])

# Find the best sequence and its score
best_score = df['ml_score'].max()
best_sequence = df.loc[df['ml_score'].idxmax(), 'sequences']

# Print the unique sequences found
print(f"There were {len(set(df['sequences']))} unique sequences found.")

# Print the best sequence and its score
print(f"Best Sequence:")
print(best_sequence)

print(f"Best Score: {best_score:.4f}")

# Print the percentiles
print(f"Percentiles:")
print(f"  25th Percentile: {percentiles[0]:.4f}")
print(f"  50th Percentile: {percentiles[1]:.4f}")
print(f"  75th Percentile: {percentiles[2]:.4f}")
print(f"  90th Percentile: {percentiles[3]:.4f}")
print(f"  99th Percentile: {percentiles[4]:.4f}")

# Plotting the percentiles
#plt.figure(figsize=(10, 6))
#plt.plot([25, 50, 75, 90, 99], percentiles, marker='o', linestyle='--', color='b')
#plt.xlabel('Percentile')
#plt.ylabel('Score')
#plt.title('Percentiles of Scores')
#plt.grid(True)
#plt.show()

# Plot the score distribution of the top 10,000 sequences
top_n = 20000
top_10000_df = df.nlargest(top_n, 'ml_score')

plt.figure(figsize=(10, 6))
plt.hist(top_10000_df['ml_score'], bins=50, alpha=0.75, color='g', edgecolor='black')
plt.axvline(best_score, color='r', linestyle='dashed', linewidth=1, label=f'Best Score: {best_score:.4f}')
plt.axvline(optimizer.wt_score, color='k', linestyle='dashed', linewidth=1, label=f'WT Score: {float(optimizer.wt_score):.4f}')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.title(f'Distribution of Scores (Top {top_n:,} Sequences)')
plt.legend()
plt.grid(True)
plt.savefig(f'Distribution of Scores (Top {top_n:,} Sequences)', bbox_inches='tight', dpi=200)
plt.show()
